In [1]:
!pip install confluent_kafka sseclient

In [2]:
from confluent_kafka import SerializingProducer, DeserializingConsumer
from confluent_kafka.serialization import StringSerializer, StringDeserializer
from confluent_kafka.admin import AdminClient, NewTopic
from uuid import uuid4
import sys, random

brokers = "kafka1:9092,kafka2:9093"

In [3]:
pconf = {
    'bootstrap.servers': brokers,
    'partitioner': 'murmur2_random',
    'key.serializer': StringSerializer('utf_8'),
    'value.serializer':  StringSerializer('utf_8')
}

In [4]:
p = SerializingProducer(pconf)

In [5]:
stocks = "stocks_finale.csv"

In [6]:
from datetime import datetime
  #2023-10-13T08:16:13Z
def construct_stock(row):
    time_stamp = time.time()
    date_time = datetime.fromtimestamp(time_stamp)
    str_date_time = date_time.strftime("%Y-%m-%dT%H:%M:%SZ") #"%d-%m-%Y, %H:%M:%S"
    stock = {"name": row[1],
             "price": float(row[0]),
             "timestamp":str_date_time
             }
    return stock

In [ ]:
import csv, json
import time
n=0
with open(stocks) as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=';')
    next(csv_reader, None)
    try:
        for row in csv_reader:
            if n == 30000:
                break
            stock = construct_stock(row)
            print(stock)
            p.produce('stock', value=json.dumps(stock))
            p.poll(0)
            p.flush()
            time.sleep(0.16)
            n = n + 1
    except BufferError:
        sys.stderr.write('%% Local producer queue is full (%d messages awaiting delivery): try again\n' % len(p))







{'name': 'ISRG', 'price': 191.9256, 'timestamp': '2023-11-06T19:59:17Z'}
{'name': 'BBT', 'price': 30.67, 'timestamp': '2023-11-06T19:59:18Z'}
{'name': 'FITB', 'price': 16.54, 'timestamp': '2023-11-06T19:59:18Z'}
{'name': 'ZION', 'price': 24.28, 'timestamp': '2023-11-06T19:59:18Z'}
{'name': 'HCP', 'price': 46.97, 'timestamp': '2023-11-06T19:59:18Z'}
{'name': 'LLY', 'price': 53.74, 'timestamp': '2023-11-06T19:59:19Z'}
{'name': 'MAC', 'price': 61.06, 'timestamp': '2023-11-06T19:59:19Z'}
{'name': 'ADBE', 'price': 38.9, 'timestamp': '2023-11-06T19:59:19Z'}
{'name': 'AZO', 'price': 382.14, 'timestamp': '2023-11-06T19:59:19Z'}
{'name': 'CL', 'price': 54.135, 'timestamp': '2023-11-06T19:59:19Z'}
{'name': 'PPG', 'price': 68.51, 'timestamp': '2023-11-06T19:59:20Z'}
{'name': 'KMB', 'price': 89.85, 'timestamp': '2023-11-06T19:59:20Z'}
{'name': 'AMAT', 'price': 13.86, 'timestamp': '2023-11-06T19:59:20Z'}
{'name': 'KR', 'price': 14.155, 'timestamp': '2023-11-06T19:59:20Z'}
{'name': 'WEC', 'price': 3

### Use only if you need to purge all the messages in the queue

In [ ]:
brokers = "kafka1:9092,kafka2:9093"

admin_client = AdminClient({"bootstrap.servers":brokers})
admin_client.delete_topics(topics=['stock'])